In [1]:
print("teste")

teste


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark-etl").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/05 14:43:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
